Para el correcto funcionamiento de este fichero es necesario crear dentro de Colab las carpetas ods, csv, txt y pdf. Dentro de las carpetas pdf y ods deberán estar todos los ficheros a transformar

In [ ]:
import pandas as pd
!pip install tika
from tika import parser
from google.colab import files
!pip install odfpy

In [12]:
def init():

  # opening pdf file
  parsed_pdf = parser.from_file(ruta + "pdf/" + document + ".pdf")

  # saving content of pdf
  # you can also bring text only, by parsed_pdf['text']
  # parsed_pdf['content'] returns string
  data = parsed_pdf['content']

  # writing to file
  with open(ruta + "txt/" + document + ".txt", 'w') as f:
      f.write(data)

  # reading from file
  file1 = open(ruta + "txt/" + document + ".txt", 'r')
  Lines = file1.readlines()
  extractPages(pagina, pages, Lines)
  extractPage([], pages, 1, Lines)

In [3]:
# Crea listas de cada página
def extractPages(pagina, pages, Lines):
  for line in Lines:
    if line.find("de Alicante CONTRATOS FORMALIZADOS") != -1:
      pages +=1
      if pages !=0:
        extractDataFormalizados(pagina, pages)
        pagina = []
    if line != '\n' and line != ' \n':
      pagina.append(line)
  extractDataFormalizados(pagina, pages+1)

In [10]:
# Extraemos la primera página para poder controlar el formato
def extractPage(myList1, pages1, page1, Lines):
  for line in Lines:
    if line.find("de Alicante") != -1:
      pages1 +=1
      if pages1 == page1:
        writeTxt(myList1,page1)
        break;
      else:
        myList1 = []
    if line != '\n' and line != ' \n':
      myList1.append(line)

In [11]:

def writeTxt(data_List, page):
  data_Str = ' '.join(data_List)
  with open(ruta + "txt/" + document + "_"+ str(page) + ".txt", 'w') as f:
    f.write(data_Str)

In [4]:
# Extrae los datos de una página de los contratos formalizados guardada en una lista de Strings

def extractDataFormalizados(pagina, pages):

  # Inicializamos las variables para poder usarlas en cada llamada e introducirlas en el DataSet tras el bucle For
  numExpediente = objContrato = tipoTramitacion = tipoContrato = procContratacion = plazoEjecucion = presupLicitacion = ""
  idContratista = razonSocial = importeAdjudicacionSin = importeTotalAdjudicacion = fechaFormalizacion = organismo = ""
  formalizacion = False

  # print("Aqui se extraerían los datos de la página " + str(pages))
  for count, value in enumerate(pagina):
    if "Nº Expediente:" in value:
      organismo = value[(value.find("Órgano gestor:") + 15):(value.find("Nº Expediente:"))]
      numExpediente = value[(value.find("Nº Expediente:")) + 15:].replace('\n', '')
    if "Página:" in value:
      idObjContrato = count + 1
      if anyo != 2015:
        idObjContrato += 1
      objContrato = pagina[idObjContrato]
      while ("Objeto del contrato:" not in pagina[idObjContrato+1]):
        idObjContrato += 1
        linea2 = pagina[idObjContrato]
        objContrato += linea2
        objContrato = objContrato.replace('\n', " ")
    if "Tramitación:" in value:
      tipoTramitacion = value[(value.find("Tramitación:")) + 13:]
      if "Tipo de" in tipoTramitacion:
        tipoTramitacion = tipoTramitacion[:(tipoTramitacion.find("Tipo de"))]
      tipoContrato = value[:(value.find("Tramitación:"))-1]
      presupLicitacion = (pagina[count+1]).replace('\n', '')
      if presupLicitacion[0].isdigit() == False:
          presupLicitacion =""
    if "Plazo de ejecuci" in value:
      idContratacion = count - 1
      while "Negociado" not in pagina[idContratacion] and "Abierto" not in pagina[idContratacion] and "Excepcional" not in pagina[idContratacion]:
        idContratacion -= 1
      procContratacion = (pagina[idContratacion]).replace('\n', '')
      if "Importe" not in pagina[idContratacion -1 ]:
        razonSocial = (pagina[idContratacion -1 ]).replace('\n', '')
      plazoEjecucion = ""
      if( idContratacion+1 < count):
        for x in range(idContratacion+1, count+1):
          plazoEjecucion += pagina[x]
      plazoEjecucion += value[:(value.find("Plazo"))]
      if "Plazo de ejecu" in plazoEjecucion:
        plazoEjecucion = plazoEjecucion[:(plazoEjecucion.find("Plazo"))].replace('\n', '')
    if "C.I.F. / N.I.F. :" in value:
      idContratista = value[(value.find("C.I.F.")) + 17 : (value.find("Nacionalidad"))]
    if "Importe adjudicación sin I.V.A.:" in value:
      importeAdjudicacionSin = value[(value.find("sin I.V.A.:")) + 12 : (value.find("Importe adjudicación con I.V.A.:"))]
      importeTotalAdjudicacion = value[(value.find("con I.V.A.:")) + 12 :]
    if "Fecha formalización:" in value and formalizacion == False:
      fechaFormalizacion = value[(value.find("Fecha formalización:")) + 21 :]
      formalizacion = True

  # Creamos la lista fila con todos los datos de una fila del DataSet e introducimos la fila en el fichero, esto se repite en los x llamadas que se hacen a la función,
  # x equivalente al número de páginas del documento
  fila = [numExpediente, objContrato, organismo, tipoTramitacion, tipoContrato, procContratacion, presupLicitacion, idContratista, razonSocial, importeAdjudicacionSin, importeTotalAdjudicacion, plazoEjecucion, fechaFormalizacion]
  formalizados.loc[len(formalizados)] = fila
  # formalizados.to_csv(ruta + "csv/formalizados_" + str(anyo) + ".csv")
  formalizados.to_csv(ruta + "csv/formalizados_" + str(anyo) + ".csv", encoding='utf-8')



In [5]:
# Convierte los archivos ODS a CSV para poder juntar todos en el mismo formato
def ods_to_csv():
  for anyo in range (2019, 2023):
    columnas = ["Num. Exp",
                "Objeto Contrato",
                "Organismo",
                "Tipo Tramitación",
                "Tipo de Contrato",
                "Procedimiento de Contratación",
                "Importe Total Presup. Licit.",
                "NIF/CIF Adjudicatario/Contratista",
                "Razón Social Adjudicatario/Contratista",
                "Importe Adjudicación Sin Impuestos",
                "Importe total Adjudicación",
                "Plazo Ejecución",
                "Fecha Formalización Contrato"]

    columnasODS = ["Num. Exp",
                  "Objeto Contrato",
                  "Fecha Formalización Contrato",
                  "Tipo de Contrato",
                  "Procedimiento de Contratación",
                  "Importe Total Presup. Licit.",
                  "Razón Social Adjudicatario/Contratista",
                  "Importe total Adjudicación",
                  "Plazo Ejecución",
                  "Borrar"]

    ruta = "/content/"
    document = "ods/formalizados_" + str(anyo) + ".ods"

    if anyo == 2021:
      formalizados = pd.read_excel(ruta + document, engine="odf", header=3)
    if anyo == 2019 or anyo == 2020:
      formalizados = pd.read_excel(ruta + document, engine="odf", header=1)
      formalizados = formalizados.drop(0)
    if anyo == 2022:
      formalizados = pd.read_excel(ruta + document, engine="odf", header=2)
      formalizados = formalizados.drop(0)

    if anyo == 2020:
      formalizados = formalizados.drop(columns="Unnamed: 3")
    if anyo == 2021:
      formalizados = formalizados.drop(columns="Unnamed: 3")
      formalizados = formalizados.drop(columns="Unnamed: 5")
      formalizados = formalizados.drop(columns="Unnamed: 11")
      columnasODS = columnas
      columnasODS.extend(["Borrar", "Borrar", "Borrar"])
      formalizados.columns = columnasODS
      formalizados = formalizados.drop(columns="Borrar")

    if anyo == 2019 or anyo == 2020:
      formalizados.columns = columnasODS
      formalizados = formalizados.drop(columns="Borrar")
      formalizados.insert(0, "Organismo", "NaN")
      formalizados.insert(0, "Tipo Tramitación", "NaN")
      formalizados.insert(0, "NIF/CIF Adjudicatario/Contratista", "NaN")
      formalizados.insert(0, "Importe Adjudicación Sin Impuestos", "NaN")
      formalizados=formalizados.reindex(columns=columnas)

    if anyo == 2022:
      formalizados = formalizados.drop(columns="Descripción Lotes Contrato")
      formalizados.insert(3, "Organismo", "NaN")
      formalizados.insert(5, "Tipo de Contrato", "NaN")
      formalizados.insert(10, "Importe Adjudicación Sin Impuestos", "NaN")
      columnas2022 = ['Num. Exp', 'Objeto Contrato', 'Tipo Tramitación', 'Organismo',
                      'Procedimiento de Contratación', 'Tipo de Contrato',
                      'Importe Total Presup. Licit.',
                      'Importe total Adjudicación', 'Fecha Formalización Contrato',
                      'Plazo Ejecución', 'Importe Adjudicación Sin Impuestos',
                      'NIF/CIF Adjudicatario/Contratista',
                      'Razón Social Adjudicatario/Contratista']
      formalizados.columns = columnas2022
      formalizados=formalizados.reindex(columns=columnas)

    # formalizados.to_csv(ruta + "csv/formalizados_" + str(anyo) + ".csv")
    formalizados.to_csv(ruta + "csv/formalizados_" + str(anyo) + ".csv", encoding='utf-8')

In [6]:
# Une los contratos de todos los años, añadiendo una columna "Año" en un archivo CSV
def unir_formalizados():
  union = pd.DataFrame()
  for anyo in range (2015, 2023):
    ruta = "/content/"
    document = "csv/formalizados_" + str(anyo) + ".csv"

    formalizados = pd.read_csv(ruta+document)
    formalizados = formalizados.drop(columns="Unnamed: 0")

    formalizados.insert(len(columnas), "Año", anyo)
    union = pd.concat([union, formalizados])


    # display(union)
  # union.to_csv(ruta + "formalizados" + ".csv")
  union.to_csv(ruta + "formalizados" + ".csv", encoding='utf-8')


In [13]:
# Ejecutar todo
for anyo in range (2015, 2019):
  pagina = []
  pages = -1
  columnas = ["Num. Exp",
              "Objeto Contrato",
              "Organismo",
              "Tipo Tramitación",
              "Tipo de Contrato",
              "Procedimiento de Contratación",
              "Importe Total Presup. Licit.",
              "NIF/CIF Adjudicatario/Contratista",
              "Razón Social Adjudicatario/Contratista",
              "Importe Adjudicación Sin Impuestos",
              "Importe total Adjudicación",
              "Plazo Ejecución",
              "Fecha Formalización Contrato"]
  formalizados = pd.DataFrame(columns = columnas)
  ruta = "/content/"
  document = "contratos-formalizados-" + str(anyo);
  init()

ods_to_csv()

In [8]:
unir_formalizados()

In [9]:
for x in range (2015, 2023):
  files.download("/content/csv/formalizados_" + str(x) + ".csv")
files.download("/content/formalizados" + ".csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>